<a href="https://colab.research.google.com/github/farrukh602/BirdCLEF2024-Classification-base-on-sounds/blob/main/AudioProcessing/process_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import os
import librosa
import librosa.display
import IPython.display as ipd
from itertools import cycle
from tqdm import tqdm

# Configuration

In [4]:
class Configuration:

  recompute_dataframes=False

  class Paths:
    train_audio_folders = '/content/drive/MyDrive/Kaggle/Bird_CLEF2024/train_audio'
    train_audio_files='/content/drive/MyDrive/Kaggle/Bird_CLEF2024/train_audio/*/*.ogg'
    unlabeld_soundspaces='/content/drive/MyDrive/Kaggle/Bird_CLEF2024/unlabeled_soundscapes'
    taxonomy_birds = '/content/drive/MyDrive/Kaggle/Bird_CLEF2024/eBird_Taxonomy_v2021.csv'
    meta_data='/content/drive/MyDrive/Kaggle/Bird_CLEF2024/train_metadata.csv'
    sample_submission='/content/drive/MyDrive/Kaggle/Bird_CLEF2024/sample_submission.csv'


  class ClassInfo:
    class_names=sorted(os.listdir('/content/drive/MyDrive/Kaggle/Bird_CLEF2024/train_audio'))
    num_classes=len(class_names)
    labels=list(range(num_classes))
    labels_with_names = dict(zip(labels,class_names))

config=Configuration()

# Extracting audio signals and sample rates for audio processing

In [5]:
# Code for extracting the audio signal and sample rate of single example of each bird species
meta_data=pd.read_csv(config.Paths.meta_data)
meta_data['filepath'] = config.Paths.train_audio_folders+'/'+meta_data.filename
if config.recompute_dataframes:
  birds=meta_data['primary_label'].unique()
  df= meta_data.set_index('primary_label')
  bird_species=[]
  audio_signal=[]
  sample_rate=[]
  audio_data={}
  for bird in tqdm(birds, desc='Loading audio signal and sample rate'):

    path = df.loc[bird].iloc[0,:]['filepath']
    audio, sr = librosa.load(path)
    bird_species.append(bird)
    audio_signal.append(audio)
    sample_rate.append(sr)

  audio_data={'bird_species':bird_species,'audio_signal':audio_signal,'sample_rate':sample_rate}
  pd.DataFrame(audio_data).to_json('/content/drive/MyDrive/Kaggle/Bird_CLEF2024/audio_data.json')
else:
  audio_data = pd.read_json('/content/drive/MyDrive/Kaggle/Bird_CLEF2024/audio_data.json')

In [ ]:
# Code for extracting the audio signal and sample rate of all the examples of each bird species
bird_species=[]
audio_signal=[]
sample_rate=[]
audio_data={}
audio, sr = 0,0
for bird in birds:
  if bird.startswith('c'):

    for i in tqdm(range(df.loc[bird].shape[0]),desc=f'Loading audio signal and sample rate for {bird}'):
      path = df.loc[bird].iloc[0,:]['filepath']
      audio, sr = librosa.load(path)
      bird_species.append(bird)
      audio_signal.append(audio)
      sample_rate.append(sr)

    globals()[f'audio_data_{bird[0]}']={'bird_species':bird_species,'audio_signal':audio_signal,'sample_rate':sample_rate}
  else:
    continue

# Foundational details about audio files

In [18]:
len(audio_data['audio_signal'][0])

603073

In [20]:
audio_data['duration_per_sample']=1/audio_data['sample_rate']
audio_data['total_duration']=audio_data['audio_signal'].apply(lambda x: len(x)*0.000045)
audio_data

,bird_species,audio_signal,sample_rate,duration_per_sample,total_duration
0,asbfly,"[1.1239e-06, -3.573e-07, -2.4089e-06, 4.2735e-...",22050,0.000045,27.138285
1,ashdro1,"[-5.3512000000000005e-06, -1.4844300000000001e...",22050,0.000045,59.382765
2,ashpri1,"[-3.243e-07, 1.5030000000000002e-07, 1.6685e-0...",22050,0.000045,25.142445
3,ashwoo2,"[-1.83616e-05, 6.191e-07, -9.752800000000001e-...",22050,0.000045,49.870125
4,asikoe2,"[8.736e-07, 1.3690000000000001e-06, 1.4408e-06...",22050,0.000045,22.213440
...,...,...,...,...,...
177,woosan,"[2.1520000000000003e-06, -2.0713e-06, 2.8441e-...",22050,0.000045,25.842285
178,wynlau1,"[-2.3569000000000003e-06, -7.584e-07, 1.5648e-...",22050,0.000045,102.461760
179,yebbab1,"[5.9283000000000005e-06, 1.6579100000000002e-0...",22050,0.000045,9.979245
180,yebbul3,"[-2.00361e-05, -7.918000000000001e-06, 2.41159...",22050,0.000045,39.631680


0      0.000045
1      0.000045
2      0.000045
3      0.000045
4      0.000045
         ...   
177    0.000045
178    0.000045
179    0.000045
180    0.000045
181    0.000045
Name: sample_rate, Length: 182, dtype: float64